<a href="https://colab.research.google.com/github/mirpouya/TensorFlow-Tutorial/blob/main/Convolutional_Neural_Net_Malaria_Detection_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds

In [2]:
dataset, dataset_info = tfds.load("malaria", with_info=True, as_supervised=True, shuffle_files=True, split=["train"])

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/27558 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/malaria/1.0.0.incompleteGQW2O3/malaria-train.tfrecord*...:   0%|          …

Dataset malaria downloaded and prepared to /root/tensorflow_datasets/malaria/1.0.0. Subsequent calls will reuse this data.


In [3]:
dataset

[<_PrefetchDataset element_spec=(TensorSpec(shape=(None, None, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>]

In [4]:
type(dataset)

list

In [5]:
len(dataset)

1

In [6]:
len(dataset[0])

27558

In [7]:
print(dataset[0])

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, None, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>


In [8]:
dataset_info

tfds.core.DatasetInfo(
    name='malaria',
    full_name='malaria/1.0.0',
    description="""
    The Malaria dataset contains a total of 27,558 cell images with equal instances
    of parasitized and uninfected cells from the thin blood smear slide images of
    segmented cells.
    """,
    homepage='https://lhncbc.nlm.nih.gov/publication/pub9932',
    data_path=PosixGPath('/tmp/tmpfl5tspl4tfds'),
    file_format=tfrecord,
    download_size=337.08 MiB,
    dataset_size=317.62 MiB,
    features=FeaturesDict({
        'image': Image(shape=(None, None, 3), dtype=uint8),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=2),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    splits={
        'train': <SplitInfo num_examples=27558, num_shards=4>,
    },
    citation="""@article{rajaraman2018pre,
      title={Pre-trained convolutional neural networks as feature extractors toward
      improved malaria parasite detection in thin blood smear images},

Models hyper parameters

In [9]:
TRAIN_RATIO = 0.7
VAL_RATIO = 0.2
TEST_RATIO = 0.1

Splitting dataset

In [10]:
def splits(dataset, TRAIN_RATIO, VAL_RATIO, TEST_RATIO):
  size = len(dataset)

  train_dataset = dataset.take(int(TRAIN_RATIO * size))
  val_dataset = dataset.skip(int(TRAIN_RATIO * size)).take(int(VAL_RATIO * size))
  test_dataset = dataset.skip(int((TRAIN_RATIO + VAL_RATIO) * size))

  return train_dataset, val_dataset, test_dataset

splitting train, validation, and test

In [11]:
train_dataset, val_dataset, test_dataset = splits(dataset[0], TRAIN_RATIO, VAL_RATIO, TEST_RATIO)

Resize and rescaling images

In [12]:
IM_SIZE = 224
def resize_rescale(image, label, img_size = IM_SIZE):
  return tf.image.resize(image, (IM_SIZE, IM_SIZE)) / 255.0, label

resize and rescalling train, validation, and test

In [13]:
train_dataset = train_dataset.map(resize_rescale)
val_dataset = val_dataset.map(resize_rescale)
test_dataset = test_dataset.map(resize_rescale)

In [14]:
train_dataset, val_dataset, test_dataset

(<_MapDataset element_spec=(TensorSpec(shape=(224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>,
 <_MapDataset element_spec=(TensorSpec(shape=(224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>,
 <_MapDataset element_spec=(TensorSpec(shape=(224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>)

In [15]:
type(train_dataset)

tensorflow.python.data.ops.map_op._MapDataset

In [16]:
train_dataset.take(1)

<_TakeDataset element_spec=(TensorSpec(shape=(224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [17]:
type(train_dataset.take(1))

tensorflow.python.data.ops.take_op._TakeDataset

In [18]:
for image, label in train_dataset.take(1):
  print(image, label)

tf.Tensor(
[[[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 ...

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]], shape=(224, 224, 3), dtype=float32) tf.Tensor(1, shape=(), dtype=int64)


shuffling train, validation, and test sets

In [19]:
BATCH_SIZE = 32

train_dataset = train_dataset.shuffle(buffer_size=8, reshuffle_each_iteration=True).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.shuffle(buffer_size=8, reshuffle_each_iteration=True).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.shuffle(buffer_size=8, reshuffle_each_iteration=True).batch(1).prefetch(tf.data.AUTOTUNE)

In [20]:
train_dataset, val_dataset, test_dataset

(<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>,
 <_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>,
 <_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>)

Model creation

In [21]:
from tensorflow.keras.layers import InputLayer, Conv2D, Dense, BatchNormalization, MaxPool2D, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy

In [22]:
from keras.api._v2.keras import activations
model = tf.keras.Sequential([
    InputLayer(input_shape = (IM_SIZE, IM_SIZE, 3)),

    Conv2D(filters= 6, kernel_size= 5, strides= 1, padding= "valid",activation= "relu"),
    BatchNormalization(),
    MaxPool2D(pool_size= 2, strides= 2),

    Conv2D(filters=16, kernel_size=5, strides=1, padding="valid", activation="relu"),
    BatchNormalization(),
    MaxPool2D(pool_size=2, strides=2),

    Flatten(),

    Dense(100, activation="relu"),
    BatchNormalization(),

    Dense(10, activation="relu"),
    BatchNormalization(),

    Dense(1, activation="sigmoid")
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 220, 220, 6)       456       
                                                                 
 batch_normalization (Batch  (None, 220, 220, 6)       24        
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 110, 110, 6)       0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 106, 106, 16)      2416      
                                                                 
 batch_normalization_1 (Bat  (None, 106, 106, 16)      64        
 chNormalization)                                                
                                                        

Compiling the model: setting optimizar, loss, and metrics

In [23]:
model.compile(optimizer = Adam(learning_rate=0.08),
              loss=BinaryCrossentropy(),
              metrics="accuracy")

fitting the model

In [24]:
history = model.fit(train_dataset, validation_data = val_dataset, epochs=3, verbose=1)

Epoch 1/3
603/603 [==============================] - 59s 74ms/step - loss: 0.6203 - accuracy: 0.6611 - val_loss: 0.6046 - val_accuracy: 0.7565
Epoch 2/3
603/603 [==============================] - 43s 71ms/step - loss: 0.3051 - accuracy: 0.8931 - val_loss: 0.2839 - val_accuracy: 0.9133
Epoch 3/3
603/603 [==============================] - 47s 78ms/step - loss: 0.2694 - accuracy: 0.9097 - val_loss: 0.3197 - val_accuracy: 0.8768


In [25]:
test_dataset, val_dataset, train_dataset

(<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>,
 <_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>,
 <_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>)

Evaluating the model

In [26]:
model.evaluate(test_dataset)

2756/2756 [==============================] - 23s 4ms/step - loss: 0.3474 - accuracy: 0.8654


[0.34738653898239136, 0.8653846383094788]

predicting

In [27]:
test_dataset.take(1)

<_TakeDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [28]:
model.predict(test_dataset.take(1))

1/1 [==============================] - 11s 11s/step


array([[0.9658948]], dtype=float32)

In [29]:
model.predict(test_dataset.take(1))[0]

1/1 [==============================] - 11s 11s/step


array([0.9582396], dtype=float32)

In [30]:
model.predict(test_dataset.take(1))[0][0]

1/1 [==============================] - 11s 11s/step


0.02706878

Saving the model

In [31]:
model.save("model")

Saving the model in Google Drive

In [32]:
from google.colab import drive

In [33]:
drive.mount("/content/drive/")

Mounted at /content/drive/


In [34]:
!cp -r /content/model/ /content/drieve/MyDrive/models

cp: cannot create directory '/content/drieve/MyDrive/models': No such file or directory


<h2> <b> Functional API </b> </h2>

In [36]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input

In [41]:
func_api_input = Input(shape = (IM_SIZE, IM_SIZE, 3), name = "Input Image")

x = Conv2D(filters=6, kernel_size=5, strides=1, padding="valid", activation="relu")(func_api_input)
x = BatchNormalization()(x)
x = MaxPool2D(pool_size=2, strides=2)(x)

x = Conv2D(filters=16, kernel_size=5, strides=1, padding="valid", activation="relu")(x)
x = BatchNormalization()(x)
x = MaxPool2D(pool_size=2, strides=2)(x)

x = Flatten()(x)

x = Dense(100, activation="relu")(x)
x = BatchNormalization()(x)
x = Dense(10, activation="relu")(x)
x = BatchNormalization()(x)

func_api_output = Dense(1, activation="sigmoid")(x)

Lenet_model = Model(func_api_input, func_api_output, name="Lenet_model")
Lenet_model.summary()

Model: "Lenet_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input Image (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 conv2d_5 (Conv2D)           (None, 220, 220, 6)       456       
                                                                 
 batch_normalization_9 (Bat  (None, 220, 220, 6)       24        
 chNormalization)                                                
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 110, 110, 6)       0         
 g2D)                                                            
                                                                 
 conv2d_6 (Conv2D)           (None, 106, 106, 16)      2416      
                                                                 
 batch_normalization_10 (Ba  (None, 106, 106, 16)      

In [42]:
Lenet_model.compile(
    optimizer = Adam(learning_rate = 0.05),
    loss = BinaryCrossentropy(),
    metrics = "accuracy"
)

In [43]:
history = Lenet_model.fit(train_dataset, validation_data = val_dataset, epochs=3, verbose=1)

Epoch 1/3
603/603 [==============================] - 57s 88ms/step - loss: 0.6179 - accuracy: 0.6598 - val_loss: 1.6807 - val_accuracy: 0.5295
Epoch 2/3
603/603 [==============================] - 51s 84ms/step - loss: 0.2736 - accuracy: 0.9029 - val_loss: 0.3559 - val_accuracy: 0.9015
Epoch 3/3
603/603 [==============================] - 46s 76ms/step - loss: 0.2060 - accuracy: 0.9332 - val_loss: 0.1843 - val_accuracy: 0.9407


Aother way of arranging the same model

In [50]:
feature_extrac_input = Input(shape = (IM_SIZE, IM_SIZE, 3), name = "Input Image")

x = Conv2D(filters=6, kernel_size=5, strides=1, padding="valid", activation="relu")(feature_extrac_input)
x = BatchNormalization()(x)
x = MaxPool2D(pool_size=2, strides=2)(x)

x = Conv2D(filters=16, kernel_size=5, strides=1, padding="valid", activation="relu")(x)
x = BatchNormalization()(x)

feature_extract_output = MaxPool2D(pool_size=2, strides=2)(x)

feature_extract_model = Model(feature_extrac_input, feature_extract_output, name="feature_extractor_model")

feature_extract_model.summary()

Model: "feature_extractor_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input Image (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 conv2d_19 (Conv2D)          (None, 220, 220, 6)       456       
                                                                 
 batch_normalization_25 (Ba  (None, 220, 220, 6)       24        
 tchNormalization)                                               
                                                                 
 max_pooling2d_18 (MaxPooli  (None, 110, 110, 6)       0         
 ng2D)                                                           
                                                                 
 conv2d_20 (Conv2D)          (None, 106, 106, 16)      2416      
                                                                 
 batch_normalization_26 (Ba  (None, 106, 10

In [52]:
lenet_input = Input(shape = (IM_SIZE, IM_SIZE, 3), name = "Input Image")

features = feature_extract_model(lenet_input)

x = Flatten()(features)

x = Dense(100, activation="relu")(x)
x = BatchNormalization()(x)
x = Dense(10, activation="relu")(x)
x = BatchNormalization()(x)
lenet_output = Dense(1, activation="sigmoid")(x)

Lenet_model_1 = Model(lenet_input, lenet_output, name="Lenet_model_1")

Lenet_model_1.summary()

Model: "Lenet_model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input Image (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 feature_extractor_model (F  (None, 53, 53, 16)        2960      
 unctional)                                                      
                                                                 
 flatten_4 (Flatten)         (None, 44944)             0         
                                                                 
 dense_10 (Dense)            (None, 100)               4494500   
                                                                 
 batch_normalization_28 (Ba  (None, 100)               400       
 tchNormalization)                                               
                                                                 
 dense_11 (Dense)            (None, 10)              

<h2> <b> Model Subclassing </b> </h2>

In [53]:
from tensorflow.keras.layers import Layer